<a href="https://colab.research.google.com/github/Top-Nuttawat/Machine-Learning-For-Physicist/blob/main/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_RAG_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Question/Answering system over SQL data

![RAG with Structured Data](https://python.langchain.com/v0.2/assets/images/sql_usecase-d432701261f05ab69b38576093718cf3.png)

## Setup

In [ ]:
## install Library
!pip -q install pypdf sentence-transformers langchain langchain_community jq faiss-cpu tiktoken
!pip -q install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [ ]:
# prompt: download folder with gdown

!gdown --folder 1MGp7wWcJcduMm4nNmjzvN4egxxqKVq1N

Retrieving folder contents
Retrieving folder 1zcr11B0BXL1R2J2Ae9Rojg8cwGzIJIIO CSV
Processing file 1li4MJ8pOGvkVVAburUFU_vOcZrcaYIRx customer_support_tickets.csv
Processing file 1QcwBij2NEY-_C7g9mNZIWTwE2Q65UI6W Financial Statements.csv
Processing file 1aZoSyDV-Fh0924cV8TIkGIjpzDGLTdPd spotify-2023.csv
Processing file 17gevLHf1xTp4-qJRhy5qU9KNHqbBJEaM TBL3-Online-Shopping-Dataset.csv
Retrieving folder 1z4hWnVJj_zCwhA1NLS8EJfk2XYNpJwFF QA
Processing file 1Q7UB67kYhO7zjyj87sY-SyPHuUov90ku question_sample.json
Processing file 1hnxmGkrguP51exD_Po5Rj-8Eu3Ldz-SN question_sample2.json
Processing file 12lnJjDaBq5-wjxoPgNNhtP1tT8QoYP_Z question_sample3.json
Processing file 1z5YBBmoHeG2oXwETUUtwumI--CldpKeR question_sample4.json
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1li4MJ8pOGvkVVAburUFU_vOcZrcaYIRx
To: /content/dataset/CSV/customer_support_tickets.csv
100% 3.95M/3.95M [00:00<00

### install postgresql

In [ ]:
!apt-get -y install postgresql postgresql-contrib

### Start PostgreSQL service & create uset & database

### register postgresql cloud


https://console.aiven.io

In [ ]:
# Start PostgreSQL service
!service postgresql start

# Set password for the 'postgres' user
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD '1234';"

# Create a new database
!sudo -u postgres createdb mydatabase


 * Starting PostgreSQL 14 database server
   ...done.
ALTER ROLE


In [ ]:
!pip -q install psycopg2-binary sqlalchemy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.2 MB/s eta 0:00:00


### Transform csv to DB

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from google.colab import userdata

# Load CSV files into pandas DataFrames
financial_statements_df = pd.read_csv('/content/dataset/CSV/Financial Statements.csv')
online_shopping_df = pd.read_csv('/content/dataset/CSV/TBL3-Online-Shopping-Dataset.csv')
customer_support_tickets_df = pd.read_csv('/content/dataset/CSV/customer_support_tickets.csv')
spotify_data_df = pd.read_csv('/content/dataset/CSV/spotify-2023.csv', encoding='ISO-8859-1')

# Database connection parameters for local PostgreSQL
db_name = 'mydatabase'           # Your database name
db_user = 'postgres'             # PostgreSQL default superuser
db_password = '1234'             # The password you set above
db_host = 'localhost'            # Localhost for local connection
db_port = '5432'                 # Default PostgreSQL port

# Create a connection string
# connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
connection_string =  userdata.get('AIVEN_URL')
# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Insert DataFrames into PostgreSQL tables
financial_statements_df.to_sql('financial_statements', engine, if_exists='replace', index=False)
online_shopping_df.to_sql('online_shopping', engine, if_exists='replace', index=False)
customer_support_tickets_df.to_sql('customer_support_tickets', engine, if_exists='replace', index=False)
spotify_data_df.to_sql('spotify_data', engine, if_exists='replace', index=False)


TimeoutException: Requesting secret AIVEN_URL timed out. Secrets can only be fetched when running from the Colab UI.

In [ ]:
## list Table in DB
pd.read_sql("""
            SELECT schemaname, tablename, tableowner
            FROM pg_catalog.pg_tables
            WHERE schemaname != 'pg_catalog' AND schemaname!='tiger' AND
            schemaname != 'information_schema';
            """,
            engine
            )

,schemaname,tablename,tableowner
0,public,financial_statements,postgres
1,public,online_shopping,postgres
2,public,customer_support_tickets,postgres
3,public,spotify_data,postgres


## langchain

### Download Database with SQLDatabase (langchain)

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(connection_string)
print(db.dialect)
print(db.get_usable_table_names())


postgresql
['customer_support_tickets', 'financial_statements', 'online_shopping', 'spotify_data']


In [ ]:
print(db.get_table_info())


CREATE TABLE customer_support_tickets (
	"Ticket_ID" BIGINT, 
	"Customer_Name" TEXT, 
	"Customer_Email" TEXT, 
	"Customer_Age" BIGINT, 
	"Customer_Gender" TEXT, 
	"Product_Purchased" TEXT, 
	"Date_of_Purchase" TEXT, 
	"Ticket_Type" TEXT, 
	"Ticket_Subject" TEXT, 
	"Ticket_Description" TEXT, 
	"Ticket_Status" TEXT, 
	"Resolution" TEXT, 
	"Ticket_Priority" TEXT, 
	"Ticket_Channel" TEXT, 
	"First_Response_Time" TEXT, 
	"Time_to_Resolution" TEXT, 
	"Customer_Satisfaction_Rating" DOUBLE PRECISION
)

/*
3 rows from customer_support_tickets table:
Ticket_ID	Customer_Name	Customer_Email	Customer_Age	Customer_Gender	Product_Purchased	Date_of_Purchase	Ticket_Type	Ticket_Subject	Ticket_Description	Ticket_Status	Resolution	Ticket_Priority	Ticket_Channel	First_Response_Time	Time_to_Resolution	Customer_Satisfaction_Rating
1	Marisa Obrien	carrollallison@example.com	32	Other	GoPro Hero	2021-03-22	Technical issue	Product setup	I'm having an issue with the {product_purchased}. Please assist.

Your bill

In [ ]:
print(db.get_table_info(['online_shopping']))


CREATE TABLE online_shopping (
	"CustomerID" BIGINT, 
	"Gender" TEXT, 
	"Location" TEXT, 
	"Tenure_Months" BIGINT, 
	"Transaction_ID" BIGINT, 
	"Transaction_Date" TEXT, 
	"Product_SKU" TEXT, 
	"Product_Description" TEXT, 
	"Product_Category" TEXT, 
	"Quantity" BIGINT, 
	"Avg_Price" DOUBLE PRECISION, 
	"Total_Price" DOUBLE PRECISION, 
	"Delivery_Charges" DOUBLE PRECISION, 
	"Date" TEXT, 
	"Month" BIGINT
)

/*
3 rows from online_shopping table:
CustomerID	Gender	Location	Tenure_Months	Transaction_ID	Transaction_Date	Product_SKU	Product_Description	Product_Category	Quantity	Avg_Price	Total_Price	Delivery_Charges	Date	Month
17850	M	Chicago	12	16679	2019-01-01	GGOENEBJ079499	Nest Learning Thermostat 3rd Gen-USA - Stainless Steel	Nest-USA	1	153.71	153.71	6.5	1/1/2019	1
17850	M	Chicago	12	16680	2019-01-01	GGOENEBJ079499	Nest Learning Thermostat 3rd Gen-USA - Stainless Steel	Nest-USA	1	153.71	153.71	6.5	1/1/2019	1
17850	M	Chicago	12	16696	2019-01-01	GGOENEBQ078999	Nest Cam Outdoor Security Ca

In [ ]:
online_shopping_df

,CustomerID,Gender,Location,Tenure_Months,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Total_Price,Delivery_Charges,Date,Month
0,17850,M,Chicago,12,16679,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,153.71,6.5,1/1/2019,1
1,17850,M,Chicago,12,16680,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,153.71,6.5,1/1/2019,1
2,17850,M,Chicago,12,16696,2019-01-01,GGOENEBQ078999,Nest Cam Outdoor Security Camera - USA,Nest-USA,2,122.77,245.54,6.5,1/1/2019,1
3,17850,M,Chicago,12,16699,2019-01-01,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,1,81.50,81.50,6.5,1/1/2019,1
4,17850,M,Chicago,12,16700,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,153.71,6.5,1/1/2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52919,13155,F,California,8,22504,2019-03-10,GGOEGGCX056399,Gift Card - $250.00,Gift Cards,1,250.00,250.00,0.0,3/10/2019,3
52920,18077,M,Chicago,34,24250,2019-03-28,GGOEGGCX056299,Gift Card - $25.00,Gift Cards,1,25.00,25.00,0.0,3/28/2019,3
52921,16085,M,California,15,39991,2019-10-06,GGOEGOCD078399,Google Leather Perforated Journal,Notebooks & Journals,1,10.80,10.80,6.0,10/6/2019,10
52922,16085,M,California,15,39991,2019-10-06,GGOEGOCR078499,Google Spiral Leather Journal,Notebooks & Journals,1,9.60,9.60,6.0,10/6/2019,10


test sql query

In [ ]:
db.run("SELECT * FROM online_shopping LIMIT 5;")

"[(17850, 'M', 'Chicago', 12, 16679, '2019-01-01', 'GGOENEBJ079499', 'Nest Learning Thermostat 3rd Gen-USA - Stainless Steel', 'Nest-USA', 1, 153.71, 153.71, 6.5, '1/1/2019', 1), (17850, 'M', 'Chicago', 12, 16680, '2019-01-01', 'GGOENEBJ079499', 'Nest Learning Thermostat 3rd Gen-USA - Stainless Steel', 'Nest-USA', 1, 153.71, 153.71, 6.5, '1/1/2019', 1), (17850, 'M', 'Chicago', 12, 16696, '2019-01-01', 'GGOENEBQ078999', 'Nest Cam Outdoor Security Camera - USA', 'Nest-USA', 2, 122.77, 245.54, 6.5, '1/1/2019', 1), (17850, 'M', 'Chicago', 12, 16699, '2019-01-01', 'GGOENEBQ079099', 'Nest Protect Smoke + CO White Battery Alarm-USA', 'Nest-USA', 1, 81.5, 81.5, 6.5, '1/1/2019', 1), (17850, 'M', 'Chicago', 12, 16700, '2019-01-01', 'GGOENEBJ079499', 'Nest Learning Thermostat 3rd Gen-USA - Stainless Steel', 'Nest-USA', 1, 153.71, 153.71, 6.5, '1/1/2019', 1)]"

### Using groq via Langchain

**login**
https://console.groq.com/home

**model**
https://console.groq.com/docs/models

In [ ]:
!pip -q install -U langchain-groq

In [ ]:
from langchain_groq import ChatGroq
from google.colab import userdata

llm = ChatGroq(
    api_key = userdata.get('GROQ_API'),
    model= "meta-llama/llama-4-scout-17b-16e-instruct",  # meta-llama/llama-4-maverick-17b-128e-instruct    #"llama-3.1-70b-versatile"
    temperature=0,
    # other params...
)



call llm Example

In [ ]:
## Example
messages = [
    ("system", "You are a helpful translator. Translate the usersentence to Thai."),
    ("human", "I love programming."),
]
llm.invoke(messages)

AIMessage(content='ฉันชอบเขียนโปรแกรม', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 32, 'total_tokens': 37, 'completion_time': 0.011767491, 'prompt_time': 0.00333039, 'queue_time': 0.446333371, 'total_time': 0.015097881}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'stop', 'logprobs': None}, id='run-a9a16a2e-ff1e-4f7a-8c93-1e9b03cb1758-0', usage_metadata={'input_tokens': 32, 'output_tokens': 5, 'total_tokens': 37})

### Creat sql Chain

In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
print(chain.get_prompts()[0].pretty_print())


You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLR

[link text](https://)#### Step 1 SQLQuery from LLM

In [ ]:
Quesion = "มี Ticket_ID จำนวนเท่าไหร่ ที่ Date_of_Purchase ตั้งแต่ 2020-02-01 ถึง 2020-02-28"
response = chain.invoke({"question": Quesion})
response

'Question: มี Ticket_ID จำนวนเท่าไหร่ ที่ Date_of_Purchase ตั้งแต่2020-02-01 ถึง2020-02-28\n\nSQLQuery:\n```sql\nSELECT COUNT("Ticket_ID") \nFROM customer_support_tickets \nWHERE "Date_of_Purchase" BETWEEN \'2020-02-01\' AND \'2020-02-28\';\n```'

In [ ]:
print(response.split('SQLQuery:')[-1][7:-3])


SELECT COUNT("Ticket_ID") 
FROM customer_support_tickets 
WHERE "Date_of_Purchase" BETWEEN '2020-02-01' AND '2020-02-28';



#### Step 2 SQLResult from sqlquery

In [ ]:
SQL_Result=db.run(response.split('SQLQuery:')[-1][7:-3])
SQL_Result

'[(361,)]'

#### Step 3 get Answer from LLM

In [ ]:
answer_prompt =f"""
given the following user question, corresponding SQL query, and SQL result, answer the user question.

Qtionues: {Quesion}
SQL Query: {response.split('SQLQuery:')[-1][7:-3]}
SQL Result: {SQL_Result}
Answer:
"""
messages = [
    ("system", answer_prompt),
]
answer = llm.invoke(messages)
answer

AIMessage(content='มี Ticket_ID จำนวน 361 รายการ ที่ Date_of_Purchase ตั้งแต่2020-02-01 ถึง2020-02-28', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 114, 'total_tokens': 143, 'completion_time': 0.056934862, 'prompt_time': 0.00510601, 'queue_time': 0.193800532, 'total_time': 0.062040872}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'stop', 'logprobs': None}, id='run-dd600be3-a010-4503-811a-b9de3c929cfa-0', usage_metadata={'input_tokens': 114, 'output_tokens': 29, 'total_tokens': 143})

In [ ]:
answer.content

'มี Ticket_ID จำนวน 361 รายการ ที่ Date_of_Purchase ตั้งแต่2020-02-01 ถึง2020-02-28'

### Combine chain in one process

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": Quesion})

<ipython-input-34-cf898f7f3b82>:3: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


'Error: (psycopg2.errors.SyntaxError) syntax error at or near "Question"\nLINE 1: Question: มี Ticket_ID จำนวนเท่าไหร่ ที่ Date_of_Purchase ตั...\n        ^\n\n[SQL: Question: มี Ticket_ID จำนวนเท่าไหร่ ที่ Date_of_Purchase ตั้งแต่2020-02-01 ถึง2020-02-28\nSQLQuery: \n```sql\nSELECT COUNT("Ticket_ID") \nFROM customer_support_tickets \nWHERE "Date_of_Purchase" BETWEEN \'2020-02-01\' AND \'2020-02-28\';\n```]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

#### Change prompt template for Llama4

In [ ]:
from langchain_core.prompts import PromptTemplate

template = '''You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run

Answer only SQLQuery
Example :
    - SELECT COUNT(*) FROM head WHERE age > 56
    - SELECT MAX(budget_in_billions), MIN(budget_in_billions) FROM department
    - SELECT AVG(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15

Only use the following tables:
{table_info}

Question: {input}'''
prompt = PromptTemplate.from_template(template, partial_variables={
    "table_info": db.get_table_info(),
    "top_k": 5,  # Default value for top_k
})

In [ ]:
chain = create_sql_query_chain(llm, db,prompt=prompt)
print(chain.get_prompts()[0].pretty_print())

You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run

Ans

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db,prompt=prompt)
chain = write_query | execute_query
chain.invoke({"question": Quesion})

'[(361,)]'

#### QA system over SQL data

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


# Functions to print the intermediate results at each step
def print_step(label, data):
    print(f"\033[92mStep: {label}\033[0m , \033[94mOutput: {data}\033[0m")
    return data

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer only the user question (don't explain).

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)


chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    # | answer_prompt
    | (lambda inputs: print_step("Format Answer Prompt", answer_prompt.format(question=inputs["question"], query=inputs["query"], result=inputs["result"])))
    | llm
    | StrOutputParser()
)

print(chain.invoke({"question": Quesion}))


Step: Format Answer Prompt , Output: Given the following user question, corresponding SQL query, and SQL result, answer only the user question (don't explain).

Question: มี Ticket_ID จำนวนเท่าไหร่ ที่ Date_of_Purchase ตั้งแต่ 2020-02-01 ถึง 2020-02-28
SQL Query: SELECT COUNT("Ticket_ID") FROM customer_support_tickets 
WHERE "Date_of_Purchase" BETWEEN '2020-02-01' AND '2020-02-28'
SQL Result: [(361,)]
Answer: 
361


In [ ]:
print(chain.invoke({"question": Quesion},))

Step: Format Answer Prompt , Output: Given the following user question, corresponding SQL query, and SQL result, answer only the user question (don't explain).

Question: มี Ticket_ID จำนวนเท่าไหร่ ที่ Date_of_Purchase ตั้งแต่ 2020-02-01 ถึง 2020-02-28
SQL Query: SELECT COUNT("Ticket_ID") 
FROM customer_support_tickets 
WHERE "Date_of_Purchase" BETWEEN '2020-02-01' AND '2020-02-28'
SQL Result: [(361,)]
Answer: 
361


In [ ]:
print(chain.invoke({"question": "บริษัท AAPL จัดอยู่ใน Category อะไร ในปี 2022 "}))

Step: Format Answer Prompt , Output: Given the following user question, corresponding SQL query, and SQL result, answer only the user question (don't explain).

Question: บริษัท AAPL จัดอยู่ใน Category อะไร ในปี 2022 
SQL Query: SELECT "Category" FROM financial_statements WHERE "Company" = 'AAPL' AND "Year" = 2022 LIMIT 5;
SQL Result: [('IT',)]
Answer: 
IT


In [ ]:
print(chain.invoke({"question": "ผลรวมกำไรขั้นต้นของบริษัท AAPL 5 ปีย้อนหลัง (ตั้งแต่ปี 2019-2023) มากกว่า ผลรวมกำไรขั้นต้นของบริษัท AMZN 5 ปีย้อนหลัง (ตั้งแต่ปี 2019-2023) หรือไม่ กรุณาตอบเป็น 'True' หรือ 'False'"}))

Step: Format Answer Prompt , Output: Given the following user question, corresponding SQL query, and SQL result, answer only the user question (don't explain).

Question: ผลรวมกำไรขั้นต้นของบริษัท AAPL 5 ปีย้อนหลัง (ตั้งแต่ปี 2019-2023) มากกว่า ผลรวมกำไรขั้นต้นของบริษัท AMZN 5 ปีย้อนหลัง (ตั้งแต่ปี 2019-2023) หรือไม่ กรุณาตอบเป็น 'True' หรือ 'False'
SQL Query: To answer this question, we need to calculate the total gross profit for AAPL and AMZN for the years 2019-2023 and compare them.

However, there is no data for AMZN in the provided financial_statements table. Therefore, we will assume that the task is to check if the total gross profit of AAPL for the available years (2020-2022) is more than the total gross profit of any other company (which in this case would be AAPL itself since AMZN data is not available).

```sql
SELECT 
    SUM(CASE WHEN "Company" = 'AAPL' THEN "Gross Profit" ELSE 0 END) AS AAPL_Gross_Profit
FROM 
    financial_statements
WHERE 
    "Year" BETWEEN 2019 AND 2

In [ ]:
print(chain.invoke({"question": "CustomerID ที่มียอดจำนวนเงินสั่งซื้อไม่รวมค่าขนส่งมากที่สุดมาจากที่เมืองไหน"}))

Step: Format Answer Prompt , Output: Given the following user question, corresponding SQL query, and SQL result, answer only the user question (don't explain).

Question: CustomerID ที่มียอดจำนวนเงินสั่งซื้อไม่รวมค่าขนส่งมากที่สุดมาจากที่เมืองไหน
SQL Query: SELECT "Location", SUM("Total_Price") AS "Total_Amount" 
FROM online_shopping 
GROUP BY "Location" 
ORDER BY "Total_Amount" DESC 
LIMIT 5;
SQL Result: [('Chicago', 1625885.5799999286), ('California', 1442447.3099999488), ('New York', 937022.6299999756), ('New Jersey', 409666.8599999996), ('Washington DC', 255772.23999999912)]
Answer: 
Chicago


In [ ]:
print(chain.invoke({"question": "แนะนำชื่อเพลง top5 ปีล่าสุด"}))

Step: Format Answer Prompt , Output: Given the following user question, corresponding SQL query, and SQL result, answer only the user question (don't explain).

Question: แนะนำชื่อเพลง top5 ปีล่าสุด
SQL Query: SELECT track_name FROM spotify_data ORDER BY released_year DESC LIMIT 5;
SQL Result: [('Seven (feat. Latto) (Explicit Ver.)',), ('vampire',), ('LALA',), ('WHERE SHE GOES',), ('Sprinter',)]
Answer: 
เพลง top5 ปีล่าสุด ได้แก่ 
1. Seven (feat. Latto) (Explicit Ver.)
2. vampire
3. LALA
4. WHERE SHE GOES
5. Sprinter


In [ ]:
print(chain.invoke({"question": "มี Customer_Email จำนวนเท่าไหร่ ที่ลงท้ายด้วย 'example.org'"}))


Step: Format Answer Prompt , Output: Given the following user question, corresponding SQL query, and SQL result, answer only the user question (don't explain).

Question: มี Customer_Email จำนวนเท่าไหร่ ที่ลงท้ายด้วย 'example.org'
SQL Query: SELECT COUNT("Customer_Email") FROM customer_support_tickets WHERE "Customer_Email" LIKE '%@example.org'
SQL Result: [(2796,)]
Answer: 
2796


### SQL Agents

In [ ]:
!pip -q install --upgrade --quiet langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 16.2 MB/s eta 0:00:00


In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7faa80bb9950>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7faa80bb9950>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7faa80bb9950>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [ ]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

In [ ]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Ticket_Priority อะไรมีจำนวนมากที่สุด 3 อันดับ (ตอบในรูปแบบของ list โดยมีแต่ล่ะ element มี format คือ {'Ticket_Priority': ชื่อ Ticket_Priority , 'Count': จำนวน} และ เรียงลำดับจากจำนวนมากไปน้อย) ที่มี Ticket_Channel เท่ากับ 'Chat'")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_crfx', 'function': {'arguments': '{"tool_input": ""}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 995, 'total_tokens': 1011, 'completion_time': 0.037057654, 'prompt_time': 0.028986754, 'queue_time': 0.22696307, 'total_time': 0.066044408}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b145e5a2-2a5c-4e3f-b603-064321dfe989-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {'tool_input': ''}, 'id': 'call_crfx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 995, 'output_tokens': 16, 'total_tokens': 1011})]}}
----
{'tools': {'messages': [ToolMessage(content='customer_support_tickets, financial_statements, online_shopping, spotify_data', name='sql_db_list_tables', id='d3cc5611-b3fe-4fdf-8cd5-3

In [ ]:
s['agent']['messages'][0].content

"[\n    {'Ticket_Priority': 'Medium', 'Count': 551},\n    {'Ticket_Priority': 'High', 'Count': 517},\n    {'Ticket_Priority': 'Critical', 'Count': 504}\n]"

In [ ]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="มี Customer_Email จำนวนเท่าไหร่ ที่ลงท้ายด้วย 'example.org'")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_96hb', 'function': {'arguments': '{"tool_input": ""}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 950, 'total_tokens': 966, 'completion_time': 0.039744947, 'prompt_time': 0.026739674, 'queue_time': 0.267676488, 'total_time': 0.066484621}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5f82700f-cc9f-49f7-b621-b79608f32124-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {'tool_input': ''}, 'id': 'call_96hb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 950, 'output_tokens': 16, 'total_tokens': 966})]}}
----
{'tools': {'messages': [ToolMessage(content='customer_support_tickets, financial_statements, online_shopping, spotify_data', name='sql_db_list_tables', id='3600df96-a309-4e70-9be4-6b

In [ ]:
s['agent']['messages'][0].content

"มี Customer_Email จำนวน 2796 ที่ลงท้ายด้วย 'example.org'"

In [ ]:


for s in agent_executor.stream(
    {"messages": [HumanMessage(content="ผลรวมกำไรขั้นต้นของบริษัท AAPL 5 ปีย้อนหลัง (ตั้งแต่ปี 2019-2023) มากกว่า ผลรวมกำไรขั้นต้นของบริษัท AMZN 5 ปีย้อนหลัง (ตั้งแต่ปี 2019-2023) หรือไม่ กรุณาตอบเป็น 'True' หรือ 'False'")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ypa7', 'function': {'arguments': '{"tool_input": ""}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 996, 'total_tokens': 1012, 'completion_time': 0.039707323, 'prompt_time': 0.036771349, 'queue_time': 0.396794783, 'total_time': 0.076478672}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7b11ae4c-b638-41ac-a819-aa5c5b0901e1-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {'tool_input': ''}, 'id': 'call_ypa7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 996, 'output_tokens': 16, 'total_tokens': 1012})]}}
----
{'tools': {'messages': [ToolMessage(content='customer_support_tickets, financial_statements, online_shopping, spotify_data', name='sql_db_list_tables', id='997b71a4-89bc-4295-a89a-

In [ ]:
s['agent']['messages'][0].content

'ผลรวมกำไรขั้นต้นของบริษัท AAPL5 ปีย้อนหลัง (ตั้งแต่ปี2019-2023) อยู่ที่ 526966.0 และผลรวมกำไรขั้นต้นของบริษัท AMZN5 ปีย้อนหลัง (ตั้งแต่ปี2019-2023) อยู่ที่ 690373.0 \n\nผลรวมกำไรขั้นต้นของบริษัท AAPL5 ปีย้อนหลัง (ตั้งแต่ปี2019-2023) น้อยกว่า ผลรวมกำไรขั้นต้นของบริษัท AMZN5 ปีย้อนหลัง (ตั้งแต่ปี2019-2023) \n\nคำตอบคือ False'